In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
os.environ["CUDA_VISIBLE_DEVICES"]="-1"    

import tensorflow as tf 
import numpy as np
import tensorflow_datasets as tfds

tf.compat.v1.enable_eager_execution()

mnist_train = tfds.load(name="mnist", split="train").batch(100).shuffle(1024)
mnist_test = tfds.load(name="mnist", split="train").batch(100).shuffle(1024)

In [2]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)
def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return initial

def conv2d(x, W):
    # stride
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding="SAME") # 保持大小的方法 

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")  # stride 是对输入的向量的各个维度而言的

def inference(image_batch, keep_prob):
    # conv_1 layer
    with tf.name_scope("conv_layer_1"):
        W_conv1 = weight_variable([5, 5, 1, 32]) # 5*5*1*32(channel)
        b_conv1 = bias_variable([32])
        hidden_conv1 = tf.nn.relu(conv2d(image_batch, W_conv1) + b_conv1)
        hidden_pool1 = max_pool_2x2(hidden_conv1) # 14 * 14 * 32

    # conv_2 layer
    with tf.name_scope("conv_layer_2"):
        W_conv2 = weight_variable([5, 5, 32, 64]) # 5*5*32*64(channel)
        b_conv2 = bias_variable([64])
        hidden_conv2 = tf.nn.relu(conv2d(hidden_pool1, W_conv2) + b_conv2)
        hidden_pool2 = max_pool_2x2(hidden_conv2) # 7 * 7 * 64

    # FC_layer
    with tf.name_scope("FC_layer1"):
        W_fc1 = weight_variable([7 * 7 * 64, 1024])
        b_fc1 = bias_variable([1024])

        h_pool2_flat = tf.reshape(hidden_pool2, [-1, 7 * 7 * 64])
        fc1_out = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
        h_fc1_drop = tf.nn.dropout(fc1_out, keep_prob, name="drop_out")

    W_fc2 = weight_variable([1024, 10])
    b_fc2 = bias_variable([10])
    prediction = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)
    return prediction

def compute_loss(prediction, label_batch):
    cross_entropy = tf.reduce_mean(-tf.reduce_sum(label_batch * tf.log(prediction), reduction_indices=[1]))
    return cross_entropy

def compute_accuracy(precition, ys):
    correct_prediction = tf.equal(tf.argmax(prediction,1), tf.argmax(ys,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    return accuracy

def train(image_batch, label_batch,keep_prob_in):
    train_pred = inference(image_batch, keep_prob_in)
    cross_entropy = compute_loss(train_pred, label_batch)
    train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

def test(test_batch, test_label,keep_prob_in):
    test_pred = inference(test_batch, keep_prob_in)
    accuracy = compute_accuracy(test_pred, test_label)

In [3]:

for i in range(100):
    for example in mnist_train:
        img_batch, label_batch = example["image"], example["label"]
        img_batch = tf.cast(img_batch, tf.float32)
        label_batch = tf.cast(label_batch, tf.float32)
        train(img_batch, label_batch,tf.constant(0.5, tf.float32))
    if i%10 == 0:
        for example in mnist_test:
            img_batch, label_batch = example["image"], example["label"]
            img_batch = tf.cast(img_batch, tf.float32)
            label_batch = tf.cast(label_batch, tf.float32)
            accuracy = train(img_batch, label_batch, tf.constant(1.0, tf.float32))
            print(accuracy)

InvalidArgumentError: Incompatible shapes: [100] vs. [100,10] [Op:Mul] name: mul/